# Asset Manager

In [ ]:
import naas_python
import pydash

class AssetManager:
    
    def __init__(
        self,
        workspace_id,
        storage_name,
        storage_manager
    ):
        self.workspace_id = workspace_id
        self.storage_name = storage_name
        self.storage_manager = storage_manager

    def add(
        self,
        file_path,
        visibility="public",
        content_disposition="inline",
        password=None
    ):
        # Upload file in storage
        asset = None
        file_s3 = self.storage_manager.upload_file(file_path)
        
        # Create asset
        try:
            asset = naas_python.asset.create_asset(
                workspace_id=self.workspace_id,
                asset_creation = {
                    "asset_creation": {
                        "storage_name" : self.storage_name,
                        "object_name" : file_s3,
                        "visibility" : visibility,
                        "content_disposition" : content_disposition,
                        "password": password
                    }
                }
            )
            asset_url = pydash.get(asset, 'asset.url')
            print(f"✅ Asset successfully created: {asset_url}")
        except Exception as e:
            asset_id = pydash.get(asset, 'asset.id')
        
        # Update asset
        if asset is None:
            naas_python.asset.update_asset(
                workspace_id=self.workspace_id,
                asset_id=asset_id,
                asset_update= {
                    "asset_update" : {
                        "visibility" : visibility,
                        "content_disposition" : content_disposition,
                        "password": password
                    }
                }
            )
            
        file_name = file_path.split("/")[-1]
        path_prefix = file_path.split(f"/{file_name}")[0]
        self.storage_manager.pdump(path_prefix, asset, file_name)
        return asset
    
    def get(
        self,
        file_path
    ):
        # Method to patch naas-python
        file_name = file_path.split("/")[-1]
        path_prefix = file_path.split(f"/{file_name}")[0]
        asset = self.storage_manager.pload(path_prefix, file_name)
        
        # Method to be updated with new features in naas python
#         asset = naas_python.asset.get_asset(
#             workspace_id=self.workspace_id,
#             file_path=file_path
#         )
        return asset
    
    def get_asset_url(
        self,
        file_path,
    ):
        asset = self.get(file_path)
        return pydash.get(asset, 'asset.url')
        
        
    def delete(
        self,
        asset_id,
    ):
        naas_python.asset.delete_asset(
            workspace_id=self.workspace_id,
            asset_id=asset_id
        )
        
# am = AssetManager(workspace_id, storage_name, sm)